In [28]:
import pandas as pd

In [29]:
df = pd.read_csv("root2ai - Data.csv")

In [30]:
df.head()

,Text,Target
0,reserve bank forming expert committee based in...,Blockchain
1,director could play role financial system,Blockchain
2,preliminary discuss secure transaction study r...,Blockchain
3,security indeed prove essential transforming f...,Blockchain
4,bank settlement normally take three days based...,Blockchain


In [31]:
df['Target'].value_counts()

FinTech             8551
Cyber Security      2640
Bigdata             2267
Reg Tech            2206
credit reporting    1748
Blockchain          1375
Neobanks            1069
Microservices        977
Stock Trading        787
Robo Advising        737
Data Security        347
Name: Target, dtype: int64

In [32]:
df.isnull().sum()

Text      3
Target    0
dtype: int64

In [33]:
df.dropna()

,Text,Target
0,reserve bank forming expert committee based in...,Blockchain
1,director could play role financial system,Blockchain
2,preliminary discuss secure transaction study r...,Blockchain
3,security indeed prove essential transforming f...,Blockchain
4,bank settlement normally take three days based...,Blockchain
...,...,...
22699,fourth study discusses blockchain technology e...,Reg Tech
22700,book finishes stating biggest issue emerging F...,Reg Tech
22701,people culture cess,Reg Tech
22702,author challenges execu tive lead change stop ...,Reg Tech


In [34]:
df.reset_index(inplace=True)

In [35]:
df.shape

(22704, 3)

In [36]:
#Label encoding the target feature categoories
dictionary = {'FinTech':1,'Cyber Security':2,'Bigdata':3,'Reg Tech':4,'credit reporting':5,'Blockchain':6,'Neobanks':7,'Microservices':8,'Stock Trading':9,'Robo Advising':10,'Data Security':11}
df['Target_encode'] = df['Target'].map(dictionary)
df.drop('Target',axis = 1,inplace=True)

In [37]:
df.head()

,index,Text,Target_encode
0,0,reserve bank forming expert committee based in...,6
1,1,director could play role financial system,6
2,2,preliminary discuss secure transaction study r...,6
3,3,security indeed prove essential transforming f...,6
4,4,bank settlement normally take three days based...,6


In [38]:
df.drop('index',axis=1,inplace=True)

In [39]:
df.shape

(22704, 2)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22704 entries, 0 to 22703
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Text           22701 non-null  object
 1   Target_encode  22704 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 354.9+ KB


In [41]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps = PorterStemmer()
corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', str(df['Text'][i]))
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [42]:
## Applying Countvectorizer
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()

In [43]:
#Get the filter out text of dependent and independent features
X.shape

(22704, 7581)

In [44]:
y = df['Target_encode']

In [45]:
y.shape

(22704,)

In [46]:
## Divide the dataset into Train and Test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [47]:
cv.get_feature_names()[:20]

['abak',
 'abandon',
 'abash',
 'abbeel',
 'abc',
 'abduct',
 'abdullah',
 'abhijit',
 'abid',
 'abil',
 'abl',
 'abnorm',
 'aboard',
 'abolit',
 'abound',
 'abreast',
 'abroad',
 'absenc',
 'absid',
 'absolut']

In [48]:
cv.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

In [49]:
count_df = pd.DataFrame(X_train, columns=cv.get_feature_names())

In [50]:
count_df.head()

,abak,abandon,abash,abbeel,abc,abduct,abdullah,abhijit,abid,abil,...,zhou,zine,zing,zipss,zmlg,zone,zoom,zrev,zurich,zxnglqu
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# MultinomialNB Algorithm¶

In [51]:
from sklearn.naive_bayes import MultinomialNB
classifier=MultinomialNB()
from sklearn import metrics
import numpy as np
import itertools


classifier.fit(X_train, y_train)
pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.633


# Multinomial Classifier with hyperparameter

In [35]:
classifier=MultinomialNB(alpha=0.1)


previous_score=0
for alpha in np.arange(0,1,0.1):
    sub_classifier=MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train,y_train)
    y_pred=sub_classifier.predict(X_test)
    score = metrics.accuracy_score(y_test, y_pred)
    if score>previous_score:
        classifier=sub_classifier
    print("Alpha: {}, Score : {}".format(alpha,score))

C:\Anacond_Install\lib\site-packages\sklearn\naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Alpha: 0.0, Score : 0.6112371546776992
Alpha: 0.1, Score : 0.6456692913385826
Alpha: 0.2, Score : 0.6459362071266516
Alpha: 0.30000000000000004, Score : 0.6474042439610302
Alpha: 0.4, Score : 0.6483384492192713
Alpha: 0.5, Score : 0.6417990124115841
Alpha: 0.6000000000000001, Score : 0.638996396636861
Alpha: 0.7000000000000001, Score : 0.6381956492726545
Alpha: 0.8, Score : 0.6375283598024823
Alpha: 0.9, Score : 0.6341919124516215


In [55]:
pip install evalml

  Using cached evalml-0.24.2-py3-none-any.whl (6.2 MB)
  Using cached catboost-0.25.1-cp37-none-win_amd64.whl (66.9 MB)
  Using cached texttable-1.6.3-py2.py3-none-any.whl (10 kB)
  Using cached scikit_optimize-0.8.1-py2.py3-none-any.whl (101 kB)
  Using cached category_encoders-2.2.2-py2.py3-none-any.whl (80 kB)
  Using cached shap-0.39.0-cp37-cp37m-win_amd64.whl (414 kB)
  Using cached matplotlib-3.4.2-cp37-cp37m-win_amd64.whl (7.1 MB)
  Using cached sktime-0.6.1-cp37-cp37m-win_amd64.whl (4.4 MB)
Processing c:\users\stallions dey\appdata\local\pip\cache\wheels\40\e4\ca\8af24ee94c3863d620d6a52793d82930f4d1d3515a5121d495\requirements_parser-0.2.0-py3-none-any.whl
  Using cached xgboost-1.2.1-py3-none-win_amd64.whl (86.5 MB)
  Using cached pmdarima-1.8.0-cp37-cp37m-win_amd64.whl (609 kB)
  Using cached seaborn-0.11.1-py3-none-any.whl (285 kB)
  Using cached lightgbm-3.0.0-py2.py3-none-win_amd64.whl (737 kB)
  Using cached pyaml-20.4.0-py2.py3-none-any.whl (17 kB)
  Using cached slicer-0

ERROR: sktime 0.6.1 has requirement numba>=0.50, but you'll have numba 0.48.0 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\anacond_install\\lib\\site-packages\\matplotlib\\backends\\_backend_agg.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [58]:
import evalml

ModuleNotFoundError: No module named 'evalml'